# Structure Collection

In [1]:
from typing import ClassVar
from byteclasses import ByteOrder
from byteclasses.print import byteclass_info, byteclass_inspect
from byteclasses.types.collections import FixedSizeCollection, structure, member
from byteclasses.types.primitives.characters import UChar
from byteclasses.types.primitives.generics import Byte
from byteclasses.types.primitives.integers import Int8, UInt8, UInt16, UInt32
from byteclasses.types.primitives.floats import Float32

Basic Structure

In [2]:
@structure
class BasicStructure:
    """A basic structure byteclass."""

    var1: UInt8
    var2: UInt16


bs = BasicStructure()
bs.data = b"\x00\x01\x02\x03"
print(bs, len(bs), bs.data)

BasicStructure(var1=UInt8(0), var2=UInt16(770)) 4 bytearray(b'\x00\x01\x02\x03')


In [3]:
byteclass_info(bs)

                                Byteclass Info                                 
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property                  ┃ Value                                           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ type()                    │ BasicStructure                                  │
│ is_byteclass()            │ True                                            │
│ is_byteclass_collection() │ True                                            │
│ is_byteclass_primitive()  │ False                                           │
│ mro                       │ BasicStructure -> object                        │
│ len()                     │ 4                                               │
│ str()                     │ BasicStructure(var1=UInt8(0), var2=UInt16(770)) │
│ repr()                    │ BasicStructure(var1=UInt8(0), var2=UInt16(770)) │
│ .data                     │ bytearray(b'\x00\x01\x02\x03')                  │
└───────────────────────────┴─────────────────────────────────────────────────┘

In [4]:
byteclass_inspect(bs)

╭───────────────── Byteclass Inspect ──────────────────╮
│      00 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f │
│ ---------------------------------------------------- │
│ 0x0 |00|01|02 03                                     │
╰──────────────────────────────────────────────────────╯

      Legend      
┏━━━━━━━━┳━━━━━━━┓
┃ Member ┃ Value ┃
┡━━━━━━━━╇━━━━━━━┩
│ var1   │ 0     │
│ var2   │ 770   │
└────────┴───────┘

Packed Basic Structure

In [5]:
@structure(packed=True)
class PackedStructure:
    """A Packed structure byteclass."""

    var1: UInt8
    var2: UInt16


ps = PackedStructure()
print(ps, len(ps), ps.data)

PackedStructure(var1=UInt8(0), var2=UInt16(0)) 3 bytearray(b'\x00\x00\x00')


Simple Structure

In [6]:
@structure(byte_order=b"!")
class SimpleStruct():
    a:UInt8
    b:UInt16
    c:UChar
    d:Float32
    e: UInt8
ss1: FixedSizeCollection = SimpleStruct()

In [7]:
byteclass_info(ss1)

                                                  Byteclass Info                                                  
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property                  ┃ Value                                                                              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ type()                    │ SimpleStruct                                                                       │
│ is_byteclass()            │ True                                                                               │
│ is_byteclass_collection() │ True                                                                               │
│ is_byteclass_primitive()  │ False                                                                              │
│ mro                       │ SimpleStruct -> object                                                             │
│ len()                     │ 13                                                                                 │
│ str()                     │ SimpleStruct(a=UInt8(0), b=UInt16(0), c=UChar('\x00'), d=Float32(0.0), e=UInt8(0)) │
│ repr()                    │ SimpleStruct(a=UInt8(0), b=UInt16(0), c=UChar('\x00'), d=Float32(0.0), e=UInt8(0)) │
│ .data                     │ bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')                 │
└───────────────────────────┴────────────────────────────────────────────────────────────────────────────────────┘

In [8]:
byteclass_inspect(ss1)

╭───────────────── Byteclass Inspect ──────────────────╮
│      00 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f │
│ ---------------------------------------------------- │
│ 0x0 |00|00|00 00|00|00 00 00|00 00 00 00|00          │
╰──────────────────────────────────────────────────────╯

      Legend      
┏━━━━━━━━┳━━━━━━━┓
┃ Member ┃ Value ┃
┡━━━━━━━━╇━━━━━━━┩
│ a      │ 0     │
│ b      │ 0     │
│ c      │        │
│ d      │ 0.0   │
│ e      │ 0     │
└────────┴───────┘

Direct data access

In [9]:
ss1.data = b"\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xcc\x8c?\x00"
print(ss1.data)
print(ss1.a)
print(ss1.b)

bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\xcd\xcc\x8c?\x00')
0
0


In [10]:
ss2 = SimpleStruct()
ss2.a = 1
ss2.b = 2
ss2.c = "c"
ss2.d = 4
print(ss2)
print(ss2.data)

SimpleStruct(a=UInt8(1), b=UInt16(2), c=UChar('c'), d=Float32(4.0), e=UInt8(0))
bytearray(b'\x01\x00\x00\x02c\x00\x00\x00@\x80\x00\x00\x00')


Member Assignment

In [11]:
print(ss1.a.endianness)
ss1.a = 1
print(ss1.a.data)
print(ss1.data)
ss1.b = 2
print(ss1.b.data)
print(ss1.data)
ss1.c = "c"
print(ss1.c.data)
print(ss1.data)
ss1.d = 4
print(ss1.d.data)
print(ss1.data)

NET
b'\x01'
bytearray(b'\x01\x00\x00\x00\x00\x00\x00\x00\xcd\xcc\x8c?\x00')
b'\x00\x02'
bytearray(b'\x01\x00\x00\x02\x00\x00\x00\x00\xcd\xcc\x8c?\x00')
b'c'
bytearray(b'\x01\x00\x00\x02c\x00\x00\x00\xcd\xcc\x8c?\x00')
b'@\x80\x00\x00'
bytearray(b'\x01\x00\x00\x02c\x00\x00\x00@\x80\x00\x00\x00')


Index Assignment

In [12]:
ss1[1] = 6
ss1[2] = 7
ss1[3] = 8
print(ss1.data)
print(ss1)

bytearray(b'\x01\x06\x07\x08c\x00\x00\x00@\x80\x00\x00\x00')
SimpleStruct(a=UInt8(1), b=UInt16(1800), c=UChar('c'), d=Float32(4.0), e=UInt8(0))


Nested Collections

In [13]:
@structure
class InnerStruct:
    var0: ClassVar[int] = 23
    var2: Byte

In [14]:
@structure(byte_order=ByteOrder.NATIVE, packed=True)
class OuterStruct:
    var3: ClassVar[int] = 23
    var4: Int8
    var5: UInt32
    var6: Float32
    var7: InnerStruct = member(default_factory=InnerStruct)

    def test(self):
        print("test")

nested_struct = OuterStruct()

In [15]:
byteclass_info(nested_struct)

                                                  Byteclass Info                                                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Property                  ┃ Value                                                                               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ type()                    │ OuterStruct                                                                         │
│ is_byteclass()            │ True                                                                                │
│ is_byteclass_collection() │ True                                                                                │
│ is_byteclass_primitive()  │ False                                                                               │
│ mro                       │ OuterStruct -> object                                                               │
│ len()                     │ 10                                                                                  │
│ str()                     │ OuterStruct(var4=Int8(0), var5=UInt32(0), var6=Float32(0.0),                        │
│                           │ var7=InnerStruct(var2=Byte(data=b'\x00', byte_order=b'@')))                         │
│ repr()                    │ OuterStruct(var4=Int8(0), var5=UInt32(0), var6=Float32(0.0),                        │
│                           │ var7=InnerStruct(var2=Byte(data=b'\x00', byte_order=b'@')))                         │
│ .data                     │ bytearray(b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00')                              │
└───────────────────────────┴─────────────────────────────────────────────────────────────────────────────────────┘

In [16]:
byteclass_inspect(nested_struct)

╭───────────────── Byteclass Inspect ──────────────────╮
│      00 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f │
│ ---------------------------------------------------- │
│ 0x0 |00|00 00 00 00|00 00 00 00|00                   │
╰──────────────────────────────────────────────────────╯

                              Legend                              
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Member ┃ Value                                                 ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ var4   │ 0                                                     │
│ var5   │ 0                                                     │
│ var6   │ 0.0                                                   │
│ var7   │ InnerStruct(var2=Byte(data=b'\x00', byte_order=b'@')) │
└────────┴───────────────────────────────────────────────────────┘

Member Assignment

In [17]:
nested_struct.var4 = 1

In [18]:
byteclass_inspect(nested_struct)

╭───────────────── Byteclass Inspect ──────────────────╮
│      00 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f │
│ ---------------------------------------------------- │
│ 0x0 |01|00 00 00 00|00 00 00 00|00                   │
╰──────────────────────────────────────────────────────╯

                              Legend                              
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Member ┃ Value                                                 ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ var4   │ 1                                                     │
│ var5   │ 0                                                     │
│ var6   │ 0.0                                                   │
│ var7   │ InnerStruct(var2=Byte(data=b'\x00', byte_order=b'@')) │
└────────┴───────────────────────────────────────────────────────┘

Data Attribute Assignment

In [19]:
nested_struct.data = b"\xf9\xf8\xf7\xf6\xf5\xf4\xf3\xf2\xf1\xf0"

In [20]:
byteclass_inspect(nested_struct)

╭───────────────── Byteclass Inspect ──────────────────╮
│      00 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f │
│ ---------------------------------------------------- │
│ 0x0 |f9|f8 f7 f6 f5|f4 f3 f2 f1|f0                   │
╰──────────────────────────────────────────────────────╯

                              Legend                              
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Member ┃ Value                                                 ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ var4   │ -7                                                    │
│ var5   │ 4126603256                                            │
│ var6   │ -2.4060893954673178e+30                               │
│ var7   │ InnerStruct(var2=Byte(data=b'\xf0', byte_order=b'@')) │
└────────┴───────────────────────────────────────────────────────┘

Slice Assignment

In [21]:
nested_struct[2:5] = b"\x08"

In [22]:
byteclass_inspect(nested_struct)

╭───────────────── Byteclass Inspect ──────────────────╮
│      00 01 02 03 04 05 06 07 08 09 0a 0b 0c 0d 0e 0f │
│ ---------------------------------------------------- │
│ 0x0 |f9|f8 08 08 08|f4 f3 f2 f1|f0                   │
╰──────────────────────────────────────────────────────╯

                              Legend                              
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Member ┃ Value                                                 ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ var4   │ -7                                                    │
│ var5   │ 134744312                                             │
│ var6   │ -2.4060893954673178e+30                               │
│ var7   │ InnerStruct(var2=Byte(data=b'\xf0', byte_order=b'@')) │
└────────┴───────────────────────────────────────────────────────┘